In [1]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections

In [2]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [3]:
inputLines = spark.sparkContext.textFile("train_full_parsed_clean3.csv")

In [4]:
header=inputLines.filter(lambda l: "_id" in l)

In [5]:
header.collect()

['id,trip_duration,vendor_id,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,duration,passenger_count,pickup_hour,pickup_weekday,pickup_month,snow,holiday,distance,route_cooridnates']

In [5]:
linesnoheader = inputLines.subtract(header)

In [4]:
#inputLines.count()

1449432

In [18]:
#linesnoheader.count()

1449431

In [6]:
topcoord=pd.read_csv("coordinate_more_than_1000.csv")

In [7]:
coordinates_name=topcoord['coordinates'].tolist()

In [8]:
coordinates_name

['[-73.973249, 40.739237]',
 '[-73.980185, 40.71098]',
 '[-73.974467, 40.722333]',
 '[-73.957928, 40.761162]',
 '[-73.97191, 40.743309]',
 '[-73.972169, 40.72622]',
 '[-73.972049, 40.726193]',
 '[-73.973878, 40.731708]',
 '[-73.975683, 40.728454]',
 '[-73.975064, 40.735925]',
 '[-73.958666, 40.741994]',
 '[-73.960059, 40.756935]',
 '[-73.977839, 40.712347]',
 '[-73.997981, 40.708803]',
 '[-73.976221, 40.765671]',
 '[-73.97755, 40.752237]',
 '[-73.960408, 40.756384]',
 '[-73.900045, 40.757286]',
 '[-73.971872, 40.729099]',
 '[-73.980147, 40.71087]',
 '[-73.965288, 40.777995]',
 '[-73.938584, 40.738183]',
 '[-73.972354, 40.74274]',
 '[-73.968705, 40.748108]',
 '[-73.957761, 40.759913]',
 '[-73.954221, 40.76264]',
 '[-73.973969, 40.731042]',
 '[-73.844033, 40.733901]',
 '[-73.974352, 40.722276]',
 '[-73.983711, 40.785842]',
 '[-73.947441, 40.741312]',
 '[-73.968774, 40.747839]',
 '[-74.001095, 40.736564]',
 '[-73.973097, 40.738963]',
 '[-73.939732, 40.785655]',
 '[-73.8202, 40.709571]',
 

In [9]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    row_coord=[1 if i in coordinates else 0 for i in coordinates_name ]
    feature=line.split(',')
    feature.pop(0)
    feature.pop(2)
    feature=feature[:15]
    feature=[float(i) for i in feature]
    return feature+row_coord
    
    

In [10]:
coord = linesnoheader.map(topcoordinates)

In [37]:
a='id,trip_duration,vendor_id,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,duration,passenger_count,pickup_hour,pickup_weekday,pickup_month,snow,holiday,distance,route_cooridnates'
a=a.split(',')
print(a)
a.pop(0)
a.pop(2)
a[:16]

['id', 'trip_duration', 'vendor_id', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance', 'duration', 'passenger_count', 'pickup_hour', 'pickup_weekday', 'pickup_month', 'snow', 'holiday', 'distance', 'route_cooridnates']


['trip_duration',
 'vendor_id',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'great_circle_distance',
 'duration',
 'passenger_count',
 'pickup_hour',
 'pickup_weekday',
 'pickup_month',
 'snow',
 'holiday',
 'distance',
 'route_cooridnates']

In [11]:
feature=['trip_duration','vendor_id','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude',
        'great_circle_distance','duration','passenger_count','pickup_hour','pickup_weekday','pickup_month','snow','holiday',
        'distance']
colNames=feature+coordinates_name

In [12]:
train = coord.toDF(colNames)

In [38]:
print(len(train.columns))

2813


In [13]:
train.write.csv("train_full_parsed_clean4.csv")

In [ ]:
train1=train.sample(False, 0.00001, 42)

In [25]:
train1=train.toPandas()

Py4JJavaError: An error occurred while calling o84.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:261)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:257)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.foreach(SparkPlan.scala:257)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:279)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:278)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at org.apache.spark.sql.execution.SparkPlan.executeCollect(SparkPlan.scala:278)
	at org.apache.spark.sql.Dataset$$anonfun$collectToPython$1.apply$mcI$sp(Dataset.scala:2808)
	at org.apache.spark.sql.Dataset$$anonfun$co

In [ ]:
train1.to_csv('train_full_parsed_clean5.csv',index=False) 

In [24]:
train1.toPandas().to_csv('train_full_parsed_clean5.csv',index=False) 

In [36]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder

In [41]:
encoder =OneHotEncoder(inputCol="passenger_count", outputCol="indexed_passenger_count")

In [ ]:
encoder_df=encoder.transform(train)

In [39]:
(trainingData, testData) = train.randomSplit([0.7, 0.3])

In [40]:
gbt = GBTRegressor(maxIter=10)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

IllegalArgumentException: 'Field "features" does not exist.'